In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'/Users/salmanfaris/Desktop/ds-projects/new_blog/salfaris.github.io/src/posts/2023-03-05-clash-is-prime':
  os.chdir(r'/Users/salmanfaris/Desktop/ds-projects/new_blog/salfaris.github.io/src/posts/2023-03-05-clash-is-prime')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


/var/folders/6h/lf5jg1fn1h163zw3042tmtnc0000gn/T/ipykernel_82336/3462904534.py:20: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats(fig_format)


{"/Users/salmanfaris/miniforge3/envs/quant/lib/python3.10/importlib/_bootstrap.py": 1665174078.0, "/Users/salmanfaris/miniforge3/envs/quant/lib/python3.10/importlib/_bootstrap_external.py": 1665174078.0, "/Users/salmanfaris/miniforge3/envs/quant/lib/python3.10/codecs.py": 1665174074.0, "/Users/salmanfaris/miniforge3/envs/quant/lib/python3.10/encodings/aliases.py": 1665174076.0, "/Users/salmanfaris/miniforge3/envs/quant/lib/python3.10/encodings/__init__.py": 1665174076.0, "/Users/salmanfaris/miniforge3/envs/quant/lib/python3.10/encodings/utf_8.py": 1665174077.0, "/Users/salmanfaris/miniforge3/envs/quant/lib/python3.10/abc.py": 1665174074.0, "/Users/salmanfaris/miniforge3/envs/quant/lib/python3.10/io.py": 1665174074.0, "/Users/salmanfaris/miniforge3/envs/quant/lib/python3.10/stat.py": 1665174075.0, "/Users/salmanfaris/miniforge3/envs/quant/lib/python3.10/_collections_abc.py": 1665174074.0, "/Users/salmanfaris/miniforge3/envs/quant/lib/python3.10/genericpath.py": 1665174074.0, "/Users/sal

In [2]:
def is_prime_naive(n: int) -> bool:
    # 1 and any negative integer are not prime.
    if n < 2:
        return False
    # 2 is prime.
    if n == 2:
        return True
    for k in range(2, n):
        # If n is divisible by k for any k < n, n is not prime.
        if n % k == 0:
            return False
    return True

In [3]:
def is_prime_better(n: int) -> bool:
    # 1 and any negative integer are not prime.
    if n < 2:
        return False
    # 2 is prime.
    if n == 2:
        return True
    # NEW: If n is even and n is not 2, then it is not prime.
    if n % 2 == 0:
        return False
    for k in range(3, n, 2):
        # NEW: If n is divisible by odd k for any k < n, n is not prime.
        if n % k == 0:
            return False
    return True

In [4]:
def is_prime_sqrt_trick(n: int) -> bool:
    # 1 and any negative integer are not prime.
    if n < 2:
        return False
    # 2 is prime.
    if n == 2:
        return True
    # If n is even and n is not 2, then it is not prime.
    if n % 2 == 0:
        return False
    # NEW: loop until int(sqrt(n)) + 1.
    # The + 1 is to handle if n is perfect square.
    for k in range(3, int(n**.5)+1, 2):
        # If n is divisible by odd k for any k < n, n is not prime.
        if n % k == 0:
            return False
    return True

In [5]:
#| code-fold: true
#| code-summary: speedup comparison ⚡️

import time

n = 10

def comp(f: callable) -> None:
    start = time.time()
    f(n)
    print(f"Took total of {(time.time()-start):.10f} seconds using {f.__name__}")
comp(is_prime_naive)
comp(is_prime_sqrt_trick)

Took total of 0.0000009537 seconds using is_prime_naive
Took total of 0.0000000000 seconds using is_prime_sqrt_trick


In [6]:
def is_prime_sqrt_short(n: int) -> bool:
    return n == 2 or (n > 2 and n % 2 != 0 and all(n % k != 0 for k in range(3, int(n**.5)+1, 2)))

In [7]:
def is_prime_sqrt_super_short(n: int) -> bool:
    return n == 2 or (n > 2 and n % 2 > 0 and all(n % k > 0 for k in range(3, int(n**.5)+1, 2)))